**Load the required libraries**

In [ ]:
library(data.table)
library(tidyverse)
library(DBI)
library(RSQLite)
library(dbplyr)
library(lubridate)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubridate::second()

**For limiting the number of top and bottom rows of tables printed** 

In [ ]:
options(repr.matrix.max.rows=20, repr.matrix.max.cols=30) 

**Load the data**

In [ ]:
#?lakers

In [ ]:
tail(lakers)

# Task 1

## Create a 2nd table using the unique opponents 

In [ ]:
teams <- lakers %>% distinct(opponent) %>% mutate(team_city = paste(opponent, "City")) 
teams

## Create a connection

**Create a connection to a new SQLite database. Delete the existing one if exists:**

In [ ]:
if (file.exists("~/databb/temp/lakers.db")) file.remove("~/databb/temp/lakers.db")

In [ ]:
consq <- dbConnect(RSQLite::SQLite(), "~/databb/temp/lakers.db")

## Write the tables into the SQlite database

**Get the names of tables from R object**

In [ ]:
dbWriteTable(consq, "lakers", lakers, overwrite = T)
dbWriteTable(consq, "teams", teams, overwrite = T)

## Create Lazy Query Objects and Chain Operations

**Create lazy query objects for each table in the SQLite database**

In [ ]:
lakers_db <- tbl(consq, "lakers")
teams_db <- tbl(consq, "teams")

## Create other lazy query object(s) with chain operations ##

**Select relevant columns from the lakers table**

In [ ]:
lakers_sel <- lakers_db %>% select(date, opponent, type ,player ,result)
lakers_sel

**Filter the lakers table for Kobe Bryant and his missed layups**

In [ ]:
lakers_filt <- lakers_sel %>% filter(player == "Kobe Bryant" & type == "layup" & result == "missed")
lakers_filt

**Mutate the lakers table: Convert the date to a Date object**

In [ ]:
lakers_mut <- lakers_filt %>% mutate(date = as.Date(date))
lakers_mut

**Join with the teams table using the 'opponent' column**

In [ ]:
joined_data <- lakers_mut %>% left_join(teams_db, by = "opponent")
joined_data

**Group by opponent and team_city, then summarise the missed layup count per opponent for Kobe Bryant**

In [ ]:
final_query <- joined_data %>% 
    group_by(opponent, team_city) %>% 
    summarise(missed_layup = n())
final_query

**Display the final lazy query object**

In [ ]:
show_query(final_query)

**Execute all the steps and collect the results**

In [ ]:
results <- final_query %>% collect
results

## Task 2

**Create a POSIXct datetime vector**

In [ ]:
dates <- as.POSIXct(c("2025-01-01 12:00:00", 
                      "2025-06-15 08:30:00", 
                      "2025-12-31 23:59:59"), tz = "UTC")
dates

**add 10 days to each datetime**

In [ ]:
dates_plus10 <- dates + days(10)
dates_plus10

**Format the datetime to show only the date component (YYYY-MM-DD)**

In [ ]:
dates_formatted <- format(dates, "%Y-%m-%d")
dates_formatted

**Calculate the difference in days between the first and the last date**

In [ ]:
date_diff <- difftime(dates[3], dates[1], units = "days")
date_diff